In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import random
import re
from unidecode import unidecode
import time

In [2]:
import urllib3
import json
import urllib

In [3]:
today= dt.date.today()
pp_lines = pd.read_csv(f'Lines/full_lines{today.month}_{today.day}',index_col=0)
print('find these event ids')
pp_lines.loc[pp_lines.League==82].Team.unique()

find these event ids


array(['New England', 'Orlando', 'New England/Orlando', 'Salt Lake/DC',
       'DC', 'Salt Lake'], dtype=object)

In [4]:
def scrape(url):
    #get raw json data from a single event api
    response = urllib.request.urlopen(url)
    print(response.status)
    return json.loads(response.read())

In [6]:
eventids = scrape('https://ips.sportsbook.fanduel.com/stats/eventIds')
eventids[0:5]

200


['32341134', '32382653', '32429251', '32133960', '32425481']

In [7]:
def process_data(data):
    #turn raw json data most of the way to a usable df
    path = data['attachments']['markets']
    b = pd.DataFrame()
    for k in list(path.keys()):
        a = pd.json_normalize(path[k],record_path ='runners')
        c =pd.json_normalize(path[k])
        for col in ['eventId','marketName','marketTime', 'marketType']:
            a[col] = c[col].values[0]
        b = pd.concat((b,a))
    return b

In [16]:
#have to find these by hand still
events = [32389423,32389425,32389313,32389312,32389309,32389424,32389427,32389305]
events = eventids
lines = pd.DataFrame()
for i in range(len(events)):
    #get and transform data from multiple events
    id = events[i]
    fd_url = f'https://sbapi.nj.sportsbook.fanduel.com/api/event-page?_ak=FhMFpcPWXMeyZxOx&eventId={id}&tab=shots'
    try:
        data = scrape(fd_url)
        new = process_data(data)
        print(len(lines),'before')
        lines = pd.concat((lines,new))
        print(len(lines),'after')        
    except:
        print(i,id,'error')
backup = lines.copy(deep=True)
    #seems like this is still overwriting the previous ones

200
0 before
6 after
200
6 before
12 after
200
12 before
26 after
200
26 before
32 after
200
4 32425481 error
200
5 32379203 error
200
32 before
42 after
200
42 before
48 after
200
48 before
54 after
200
54 before
64 after
200
64 before
70 after
200
70 before
80 after
200
12 32425479 error
200
80 before
86 after
200
86 before
92 after
200
92 before
609 after
200
609 before
615 after
200
615 before
625 after
200
625 before
635 after
200
635 before
641 after
200
641 before
647 after
200
21 32425480 error
200
647 before
657 after
200
657 before
663 after
200
663 before
669 after
200
25 32427150 error
200
669 before
675 after
200
675 before
689 after
200
689 before
695 after
200
29 32425485 error
200
695 before
701 after
200
701 before
707 after
200
707 before
717 after
200
717 before
723 after
200
723 before
1222 after
200
1222 before
1228 after
200
1228 before
1234 after
200
1234 before
1240 after
200
1240 before
1246 after
200
1246 before
1252 after
200
1252 before
1258 after
200
1258 b

In [14]:
backup

,selectionId,handicap,runnerName,sortPriority,runnerStatus,previousWinRunnerOdds,result.type,winRunnerOdds.americanDisplayOdds.americanOdds,winRunnerOdds.americanDisplayOdds.americanOddsInt,winRunnerOdds.trueOdds.decimalOdds.decimalOdds,winRunnerOdds.trueOdds.fractionalOdds.numerator,winRunnerOdds.trueOdds.fractionalOdds.denominator,eventId,marketName,marketTime,marketType,logo,nameAbbr
0,7017916,48.5,Over,1,ACTIVE,[],OVER,-110,-110,1.909091,100,110,32341134,Total Match Points,2023-10-08T13:31:00.000Z,TOTAL_POINTS_(OVER/UNDER),NaN,NaN
1,7017917,48.5,Under,2,ACTIVE,[],UNDER,-110,-110,1.909091,100,110,32341134,Total Match Points,2023-10-08T13:31:00.000Z,TOTAL_POINTS_(OVER/UNDER),NaN,NaN
0,50207,3.5,Jacksonville Jaguars,1,ACTIVE,[],AWAY,-122,-122,1.819672,100,122,32341134,Spread,2023-10-08T13:31:00.000Z,MATCH_HANDICAP_(2-WAY),https://assets.sportsbook.fanduel.com/images/t...,JAX Jaguars
1,50203,-3.5,Buffalo Bills,2,ACTIVE,[],HOME,100,100,2.000000,100,100,32341134,Spread,2023-10-08T13:31:00.000Z,MATCH_HANDICAP_(2-WAY),https://assets.sportsbook.fanduel.com/images/t...,BUF Bills
0,50207,0.0,Jacksonville Jaguars,1,ACTIVE,[],AWAY,142,142,2.420000,142,100,32341134,Moneyline,2023-10-08T13:31:00.000Z,MONEY_LINE,https://assets.sportsbook.fanduel.com/images/t...,JAX Jaguars
1,50203,0.0,Buffalo Bills,2,ACTIVE,[],HOME,-168,-168,1.595238,100,168,32341134,Moneyline,2023-10-08T13:31:00.000Z,MONEY_LINE,https://assets.sportsbook.fanduel.com/images/t...,BUF Bills
0,7017916,47.5,Over,1,ACTIVE,[],OVER,-110,-110,1.909091,100,110,32382653,Total Match Points,2023-09-01T23:30:00.000Z,TOTAL_POINTS_(OVER/UNDER),NaN,NaN
1,7017917,47.5,Under,2,ACTIVE,[],UNDER,-110,-110,1.909091,100,110,32382653,Total Match Points,2023-09-01T23:30:00.000Z,TOTAL_POINTS_(OVER/UNDER),NaN,NaN
0,602719,-8.0,Louisville,1,ACTIVE,[],AWAY,-110,-110,1.909091,100,110,32382653,Spread,2023-09-01T23:30:00.000Z,MATCH_HANDICAP_(2-WAY),https://assets.sportsbook.fanduel.com/images/t...,Louisville
1,588240,8.0,Georgia Tech,2,ACTIVE,[],HOME,-110,-110,1.909091,100,110,32382653,Spread,2023-09-01T23:30:00.000Z,MATCH_HANDICAP_(2-WAY),https://assets.sportsbook.fanduel.com/images/t...,GT


In [17]:
#copying data for safety and grabbing relevant columns
#wrangling the strings to turn for example 'Player To Have 2 Or More Shots' into:
#handicap :1.5   and stat: Shots


g = backup[[ 'handicap', 'runnerName',
             'winRunnerOdds.americanDisplayOdds.americanOdds', 'eventId',
             'marketName', 'marketTime']].copy(deep=True)

g['handicap'] = g.marketName.apply(lambda x: x.split('Player To Have ')[-1].split(' ')[0])
g['marketName'] = g.marketName.apply(lambda x: ' '.join(x.split('Player To Have ')[-1].split(' ')[1:]).split('Or More ')[-1])

g.columns = ['Line', 'Player','Odds', 'eventId',
       'Stat', 'marketTime']

g

,Line,Player,Odds,eventId,Stat,marketTime
0,Total,Over,-110,32341134,Match Points,2023-10-08T13:31:00.000Z
1,Total,Under,-110,32341134,Match Points,2023-10-08T13:31:00.000Z
0,Spread,Jacksonville Jaguars,-122,32341134,,2023-10-08T13:31:00.000Z
1,Spread,Buffalo Bills,100,32341134,,2023-10-08T13:31:00.000Z
0,Moneyline,Jacksonville Jaguars,142,32341134,,2023-10-08T13:31:00.000Z
...,...,...,...,...,...,...
1,Moneyline,Arizona State,740,32176469,,2023-09-23T23:30:00.000Z
0,Spread,USC,-115,32176469,,2023-09-23T23:30:00.000Z
1,Spread,Arizona State,-105,32176469,,2023-09-23T23:30:00.000Z
0,Total,Over,-110,32176469,Match Points,2023-09-23T23:30:00.000Z


In [19]:
g.Stat.unique()

array(['Match Points', '', 'Line / Total Runs Parlay', 'Runs', 'Line',
       'Margin (4-Way)', 'Hit 2+ Home Runs', 'Pitch Result - Top 3rd',
       'Record a Run', 'Record an RBI', 'Hit a Double',
       'Inning Over/Under 0.5 Runs', 'Inning Result', 'Total Runs',
       'Hit First Home Run', 'Pitch Result - Bottom 1st', 'Score',
       'Record 2+ Total Bases', 'Hit a Single',
       'Pitch Result - Bottom 3rd', 'Record a Stolen Base',
       'Run / Moneyline Parlay', 'Performance Doubles',
       '5 Innings Money Line', 'Record 2+ Hits', 'Pitch Result - Top 4th',
       '5 Innings Run Line', 'Pitch Result - Bottom 5th',
       'Diamondbacks Total Runs', 'Hit a Triple',
       'Pitch Result - Bottom 2nd', 'Guardians Total Runs',
       'Bieber - Alt Strikeouts', 'Run Lines', 'Pitch Result - Top 2nd',
       'Pitch Result - Top 1st', 'Bieber - Strikeouts', 'Record A Hit',
       'Record 3+ Hits', 'Hits to be Recorded in the 1st Inning',
       'Hit A Home Run', 'Henry - Alt Strikeouts'

In [8]:
x = pp_lines.merge(g).sort_values(by='Odds')
x

,Player,Team,Line,Stat,League,Date,Odds,eventId,marketTime
7,Khvicha Kvaratskhelia,Sampdoria,3.5,Shots,82,2023-06-04,-700,32389425,2023-06-04T16:30:00.000Z
28,Jorgen Strand Larsen,Barcelona,1.5,Shots,82,2023-06-04,-550,32389305,2023-06-04T19:00:00.000Z
1,Amir Rrahmani,Sampdoria,0.5,Shots,82,2023-06-04,-440,32389425,2023-06-04T16:30:00.000Z
14,Oihan Sancet,Real Madrid,1.5,Shots,82,2023-06-04,-380,32389313,2023-06-04T16:30:00.000Z
25,Etienne Capoue,Atletico Madrid,0.5,Shots,82,2023-06-04,-370,32389312,2023-06-04T16:30:00.000Z
2,Giacomo Raspadori,Sampdoria,2.5,Shots,82,2023-06-04,-360,32389425,2023-06-04T16:30:00.000Z
8,Piotr Zielinski,Sampdoria,2.5,Shots,82,2023-06-04,-330,32389425,2023-06-04T16:30:00.000Z
23,Angel Correa,Villarreal,1.5,Shots,82,2023-06-04,-310,32389312,2023-06-04T16:30:00.000Z
15,Brais Mendez,Sevilla,1.5,Shots,82,2023-06-04,-310,32389309,2023-06-04T16:30:00.000Z
31,Destiny Udogie,Juventus,0.5,Shots,82,2023-06-04,-290,32389424,2023-06-04T19:00:00.000Z


In [12]:
x.to_csv(f'Lines/fd_soccer{today.month}_{today.day}')

In [ ]:
https://sportsbook.fanduel.com/soccer

In [20]:
import requests
from bs4 import BeautifulSoup

def scrape_links(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            href = link.get('href')
            if href and href.startswith('http'):
                print(href)
    else:
        print("Failed to fetch the webpage.")

# Example usage
scrape_links('https://sportsbook.fanduel.com/soccer')

Failed to fetch the webpage.


In [ ]:
def scrape(url):
    #get raw json data from a single event api
    response = urllib.request.urlopen(url)
    print(response.status)
    return json.loads(response.read())

In [21]:
scrape('https://sportsbook.fanduel.com/soccer')

HTTPError: HTTP Error 403: Forbidden